In [1]:
WORKER_COUNT = 8

In [2]:
# whether to log each feature and sequence status
verbose = True

In [3]:
import gc
import os
import pandas as pd
pd.options.display.max_rows = 200
import numpy as np
import json
import datetime
import matplotlib.pyplot as plt
import itertools
import sys
sys.path.append('..')

In [18]:
# setup paths
pwd = os.getcwd().replace("notebooks","")
path_cache = pwd + 'cache/'
path_data = pwd + 'data/'

In [20]:
# setup logging
# any explicit log messages or uncaught errors to stdout and file /logs.log
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s",
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs")),
        logging.StreamHandler()
    ])
# init logger
logger = logging.getLogger()
# make logger aware of any uncaught exceptions
def handle_exception(exc_type, exc_value, exc_traceback):
    if issubclass(exc_type, KeyboardInterrupt):
        sys.__excepthook__(exc_type, exc_value, exc_traceback)
        return

    logger.error("Uncaught exception", exc_info=(exc_type, exc_value, exc_traceback))
sys.excepthook = handle_exception

In [21]:
from deepvideoclassification.architectures import Architecture
from deepvideoclassification.pretrained_CNNs import pretrained_model_names, pretrained_model_names_bucketed

# Create experiments lists and run experiments

* batch 1 = run frozen image MLP, LRCNNs and concat models on 1 of each pretrained_model_name in buckets (bucketed on feature sizes and limited to max sequence_length of 10)

* batch 2 = for best configurations from batch 1, run other pretrained models in buckets and run longer sequence lengths

* batch 3 = run trainable MLP and LRCNN on best performing frozen variants

* batch 4 = run trainable but initializing with best CNN weights

* batch 5 = run C3D models

# Batch 1

In [22]:
experiment_batch_name = 'experiment_batch_1'

In [23]:
# init list of experiments
experiments = []

In [24]:
pooling = 'max'
layer_sizes = [512, 256, 128, 0]
dropouts = [0.2]
sequence_lengths = [3,5,10]
sequence_models = ["LSTM", "SimpleRNN", "GRU", "Convolution1D"]
sequence_model_layer_counts = [1,2]

In [25]:
####################
### image_MLP_frozen 
####################

for pretrained_model_name in pretrained_model_names_bucketed:
    for layer_1_size in layer_sizes:
        for layer_2_size in layer_sizes:
            for layer_3_size in layer_sizes:
                for dropout in dropouts:

                    # build experiment parameters
                    experiment = {}
                    
                    experiment['architecture'] = 'image_MLP_frozen'
                    experiment['sequence_length'] = 1
                    experiment['pretrained_model_name'] = pretrained_model_name
                    experiment['layer_1_size'] = layer_1_size
                    experiment['layer_2_size'] = layer_2_size
                    experiment['layer_3_size'] = layer_3_size
                    experiment['dropout'] = dropout
                    experiment['pooling'] = 'max' # outperforms avg across all parameters
                    
                    # add to list of experiments
                    experiments.append(experiment)

In [26]:
####################
### video_MLP_concat
####################

for sequence_length in sequence_lengths:
    for pretrained_model_name in pretrained_model_names_bucketed:
        for layer_1_size in layer_sizes:
            for layer_2_size in layer_sizes:
                for layer_3_size in layer_sizes:
                    for dropout in dropouts:

                        # build experiment parameters
                        experiment = {}

                        experiment['architecture'] = 'video_MLP_concat'
                        experiment['pretrained_model_name'] = pretrained_model_name
                        experiment['layer_1_size'] = layer_1_size
                        experiment['layer_2_size'] = layer_2_size
                        experiment['layer_3_size'] = layer_3_size
                        experiment['dropout'] = dropout
                        experiment['pooling'] = 'max' # outperforms avg across all parameters

                        # add to list of experiments
                        experiments.append(experiment)

In [27]:
######################
### video_LRCNN_frozen
######################

for sequence_length in sequence_lengths:
    for pretrained_model_name in pretrained_model_names_bucketed:
        for layer_1_size in layer_sizes:
            for layer_2_size in layer_sizes:
                for layer_3_size in layer_sizes:
                    for dropout in dropouts:
                        for sequence_model in sequence_models:
                            for sequence_model_layers in sequence_model_layer_counts:

                                # build experiment parameters
                                experiment = {}

                                experiment['architecture'] = 'video_LRCNN_frozen'
                                experiment['pretrained_model_name'] = pretrained_model_name
                                experiment['layer_1_size'] = layer_1_size
                                experiment['layer_2_size'] = layer_2_size
                                experiment['layer_3_size'] = layer_3_size
                                experiment['dropout'] = dropout
                                experiment['pooling'] = 'max' # outperforms avg across all parameters
                                experiment['sequence_model'] = sequence_model
                                experiment['sequence_model_layers'] = sequence_model_layers

                                # add to list of experiments
                                experiments.append(experiment)

In [28]:
########################
### convert to dataframe
########################

experiments = pd.DataFrame(experiments)
experiments['model_id'] = experiments.index

In [29]:
experiments['WORKER'] = experiments['model_id'].apply(lambda x: x % WORKER_COUNT)

In [30]:
experiments.to_csv(pwd + "experiments/" + experiment_batch_name + '.csv', index=False)

In [31]:
experiments.head(100)

,architecture,dropout,layer_1_size,layer_2_size,layer_3_size,pooling,pretrained_model_name,sequence_length,sequence_model,sequence_model_layers,model_id,WORKER
0,image_MLP_frozen,0.2,512,512,512,max,inception_resnet_v2,1.0,NaN,NaN,0,0
1,image_MLP_frozen,0.2,512,512,256,max,inception_resnet_v2,1.0,NaN,NaN,1,1
2,image_MLP_frozen,0.2,512,512,128,max,inception_resnet_v2,1.0,NaN,NaN,2,2
3,image_MLP_frozen,0.2,512,512,0,max,inception_resnet_v2,1.0,NaN,NaN,3,3
4,image_MLP_frozen,0.2,512,256,512,max,inception_resnet_v2,1.0,NaN,NaN,4,4
5,image_MLP_frozen,0.2,512,256,256,max,inception_resnet_v2,1.0,NaN,NaN,5,5
6,image_MLP_frozen,0.2,512,256,128,max,inception_resnet_v2,1.0,NaN,NaN,6,6
7,image_MLP_frozen,0.2,512,256,0,max,inception_resnet_v2,1.0,NaN,NaN,7,7
8,image_MLP_frozen,0.2,512,128,512,max,inception_resnet_v2,1.0,NaN,NaN,8,0
9,image_MLP_frozen,0.2,512,128,256,max,inception_resnet_v2,1.0,NaN,NaN,9,1


# Batch 2

In [35]:
# for best configurations from batch 1, run other pretrained models 
# in buckets and run longer sequence lengths

# Batch 3

In [ ]:
# run trainable MLP and LRCNN on best performing frozen variants

In [ ]:
#######################
### image_MLP_trainable
#######################

architecture = 'video_LRCNN_trainable'

In [ ]:
#########################
### video_LRCNN_trainable
#########################

architecture = 'video_LRCNN_trainable'

# Batch 4

In [ ]:
# run trainable but initializing with best CNN weights

# Batch 5

In [ ]:
#######
### C3D
#######

architecture = 'C3D'

############
### C3Dsmall
############

architecture = 'C3Dsmall' 

# Analyze results

In [ ]:
path_models = pwd + 'models/'

results = []

for folder, subs, files in os.walk(path_models):
    for filename in files:
        if 'results.json' in filename:
            with open(os.path.abspath(os.path.join(folder, filename))) as f:
                data = json.load(f)
            results.append(data)

results = pd.DataFrame(results)        
results.sort_values("fit_val_acc", inplace=True, ascending=False)

In [ ]:
results.head(10)